In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy.stats as ss
from scipy.stats.mstats import winsorize
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
#导入做市交易企业数据
path_m = 'C:/Users/84678/Desktop/做市商效率/数据处理/新三板做市企业.xlsx'
df_n_m = pd.read_excel(path_m,sheet_name = "数据表",index_col = 0)
df_n_m_condition = pd.read_excel(path_m, sheet_name = "数据剔除（区间交易状态）",index_col = 0)

In [3]:
temp = df_n_m
for i in range(df_n_m.shape[0]):
    r = df_n_m.iloc[i]
    r1 = df_n_m_condition.iloc[i].to_frame().T
    date_b1 = r['做市前200交易日']
    date_a2 = r['做市后200交易日']
    
    if '停牌一天' in r1.loc[:,date_b1:date_a2]:
        temp = temp.drop[i]

In [4]:
df_n_m = temp

In [5]:
#定义函数，用于给四个变量分组
def divide(df):
    bins = 5
    
    try:
        df['Segment_a'] = pd.DataFrame(bins - pd.qcut(df['总资产'], bins).cat.codes)
    except:
        df['Segment_a'] = 0
        df['Segment_a'] = 4 - pd.qcut(df['总资产'].loc[df['总资产'] != 0],4).cat.codes
        df['Segment_a'][df['总资产'] == 0] = 0
        df['Segment_a'] += 1
        df['Segment_a'] = df['Segment_a'].astype(int)
    
    try:
        df['Segment_c'] = bins - pd.qcut(df['收盘价'], bins).cat.codes
    except:
        df['Segment_c'] = 0
        df['Segment_c'] = 4 - pd.qcut(df['收盘价'].loc[df['收盘价'] != 0],4).cat.codes
        df['Segment_c'][df['收盘价'] == 0] = 0
        df['Segment_c'] += 1
        df['Segment_c'] = df['Segment_c'].astype(int)
    
    try:
        df['Segment_d'] = bins - pd.qcut(df['交易天数'], bins).cat.codes
    except:
        df['Segment_d'] = 0
        df['Segment_d'] = 4 - pd.qcut(df['交易天数'].loc[df['交易天数'] != 0],4).cat.codes
        df['Segment_d'][df['交易天数'] == 0] = 0
        df['Segment_d'] += 1
        df['Segment_d'] = df['Segment_d'].astype(int)
    #     num = df.shape[0]
    #     zero, hund = (df['交易天数'] == 0).sum(), (df['交易天数'] == 100).sum()
    #     # print(f'num:{num}, zero:{zero}, hund:{hund}')
    #     if 3*zero>=num:
    #         if 2*hund>=(num-zero):
    #             df['Segment_d'] = 2
    #             df['Segment_d'][df['交易天数'] == 0] = 3
    #             df['Segment_d'][df['交易天数'] == 100] = 1
    #         else:
    #             df['Segment_d'] = 3
    #             df['Segment_d'] = 2 - pd.qcut(df['交易天数'].loc[df['交易天数'] != 0],2).cat.codes
    #             df['Segment_d'][df['交易天数'] == 0] = 3
    #     elif 3*hund>=num:
    #         if 2*zero>=(num-hund):
    #             df['Segment_d'] = 2
    #             df['Segment_d'][df['交易天数'] == 0] = 3
    #             df['Segment_d'][df['交易天数'] == 100] = 1
    #         else:
    #             df['Segment_d'] = 1
    #             df['Segment_d'] = 3 - pd.qcut(df['交易天数'].loc[df['交易天数'] != 100],2).cat.codes
    #             df['Segment_d'][df['交易天数'] == 0] = 1
    #     df['Segment_d'] = df['Segment_d'].astype(int)
                
        
    try:
        df['Segment_s'] = bins - pd.qcut(df['流通股合计'],bins).cat.codes
    except:
        df['Segment_s'] = 0
        df['Segment_s'] = 4 - pd.qcut(df['流通股合计'].loc[df['流通股合计'] != 0],4).cat.codes
        df['Segment_s'].loc[df['流通股合计'] == 0] = 0
        df['Segment_c'] += 1
        df['Segment_s'] = df['Segment_s'].astype(int)

    return df

In [6]:
divide(df_n_m)

,名称,做市首日,做市前200交易日,做市前100交易日,做市后200交易日,收盘价,交易天数,流通股合计,匹配期,总资产,做市交易前一日,Segment_a,Segment_c,Segment_d,Segment_s
代码,,,,,,,,,,,,,,,
430014.NQ,恒业世纪,2015/11/24,2015/1/27,2015/6/26,2016/9/13,3.581200,54,63218421.0,2015/6/30,2.218597e+08,2015/11/23,2,4,1,1
430021.NQ,海鑫科金,2016/1/29,2015/4/10,2015/9/1,2016/11/28,21.406700,68,135298934.0,2015/12/31,1.169655e+09,2016/1/28,1,1,1,1
430022.NQ,五岳鑫,2015/12/8,2015/2/10,2015/7/10,2016/9/29,3.002300,10,20661000.0,2015/6/30,6.642912e+07,2015/12/7,4,4,2,2
430027.NQ,北科光大,2016/3/28,2015/6/2,2015/10/30,2017/1/18,8.528400,40,21249986.0,2015/12/31,1.394838e+08,2016/3/25,3,2,1,2
430033.NQ,彩讯科技,2014/8/25,2013/11/4,2014/4/1,2015/6/23,1.842800,30,23664109.0,2014/6/30,1.615971e+08,2014/8/22,2,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836157.NQ,顺邦通信,2016/10/20,2015/12/22,2016/5/20,2017/8/11,2.127000,6,26302908.0,2016/6/30,1.325265e+08,2016/10/19,3,4,3,1
836504.NQ,博迅生物,2016/12/22,2016/3/2,2016/7/26,2017/10/20,7.830588,9,0.0,2016/6/30,1.091438e+08,2016/12/21,3,2,2,5
836680.NQ,中天新材,2016/12/26,2016/3/4,2016/7/28,2017/10/24,2.130000,1,0.0,2016/6/30,3.868163e+08,2016/12/23,1,4,5,5


In [7]:
#导入竞价交易企业数据
path_c = 'C:/Users/84678/Desktop/做市商效率/数据处理/新三板竞价企业.xlsx'
df_n_c_assets = pd.read_excel(path_c, sheet_name = "总资产",index_col = 0)
df_n_c_close = pd.read_excel(path_c, sheet_name = "收盘价",index_col = 0)
df_n_c_tradedays = pd.read_excel(path_c, sheet_name = "交易天数",index_col = 0)
df_n_c_share = pd.read_excel(path_c, sheet_name = "流通股合计",index_col = 0)
df_n_c_condition = pd.read_excel(path_c, sheet_name = "交易状态",index_col = 0)

In [23]:
df_compare = pd.DataFrame()
r = df_n_m.iloc[0]
date_b1 = r['做市前200交易日']
date_b2 = r['做市前100交易日']
date_b3 = r['做市交易前一日']
date_a1 = r['做市首日']
date_a2 = r['做市后200交易日']
date_c = r['匹配期']
temp = df_n_c_condition.loc[:,date_b1:date_a2]
temp = temp.dropna(how = 'any')
    
for j in range(temp.shape[0]):
    if '停牌一天' in temp.iloc[j]:
        temp = temp.drop[j]
            
#对每个做市交易的公司只选择temp内的竞价交易的企业
list_index = temp.index.values
df_n_c_assets1 = df_n_c_assets.loc[list_index]
df_n_c_close1 = df_n_c_close.loc[list_index] 
df_n_c_tradedays1 = df_n_c_tradedays.loc[list_index] 
df_n_c_share1 = df_n_c_share.loc[list_index]
    
#计算区间内平均收盘价，交易天数和匹配期的总资产和流通股合计
close = (df_n_c_close1.loc[ : , date_b2:date_b3]).mean(1)
tradedays_temp = (df_n_c_tradedays1.loc[:,date_b2:date_b3]).fillna(0)
tradedays = tradedays_temp.gt(0).sum(axis=1)
assets = df_n_c_assets1.loc[ : , date_c]
share = df_n_c_share1.loc[ : , date_c]
    
#将竞价企业数据集合在一个表中
df_n_c = pd.concat([assets,close,tradedays,share],axis = 1,ignore_index = True)
df_n_c.columns = ['总资产','收盘价','交易天数','流通股合计']
    
#对竞价交易的企业进行数据分组
divide(df_n_c)
    
#匹配组别相同的数据
df_n_c.reset_index(inplace = True)
divi = pd.merge(df_n_c, pd.DataFrame(r).T , on=['Segment_a','Segment_c','Segment_d','Segment_s',], how="right")
divi.dropna(inplace=True,how='any')
    
#如果匹配公司>1，则选择总资产差异最小的公司
if divi.shape[0] > 1 :
    compare = pd.DataFrame()
    compare_temp = pd.DataFrame()
    divi['diff_a'] = abs(divi['总资产_x'] - divi['总资产_y'])
    divi['diff_c'] = abs(divi['收盘价_x'] - divi['收盘价_y'])
    divi['diff_d'] = abs(divi['交易天数_x'] - divi['交易天数_y'])
    divi['diff_s'] = abs(divi['流通股合计_x'] - divi['流通股合计_y'])
        # compare = divi.loc[divi['diff'].astype(float).idxmin()].to_frame()
        # compare = compare.T
        # compare_temp['treatment index'] =  pd.DataFrame(r).T.index
        # compare_temp['control index'] = compare['代码'].values
        # df_compare = pd.concat([df_compare,compare_temp] , axis = 0)
    divi.sort_values(by=['diff_a'],inplace=True,ascending=True)
    divi.reset_index(inplace = True)
    divi.reset_index(inplace = True)
    divi['rank_a'] = divi['level_0']
    divi = divi.drop('level_0',axis=1)

    divi.sort_values(by=['diff_c'],inplace=True,ascending=True)
    divi.reset_index(inplace = True)
    divi = divi.drop('level_0',axis=1)
    divi.reset_index(inplace = True)
    divi['rank_c'] = divi['level_0']
    divi = divi.drop('level_0',axis=1)

    divi.sort_values(by=['diff_d'],inplace=True,ascending=True)
    divi.reset_index(inplace = True)
    divi = divi.drop('level_0',axis=1)
    divi.reset_index(inplace = True)
    divi['rank_d'] = divi['level_0']
    divi = divi.drop('level_0',axis=1)

    divi.sort_values(by=['diff_s'],inplace=True,ascending=True)
    divi.reset_index(inplace = True)
    divi = divi.drop('level_0',axis=1)
    divi.reset_index(inplace = True)
    divi['rank_s'] = divi['level_0']
    divi = divi.drop('level_0',axis=1)

    divi['rank'] = divi['rank_a']+divi['rank_c']+divi['rank_d']+divi['rank_s']
    divi.sort_values(by=['rank'],inplace=True,ascending=True)
    
    compare = divi.iloc[0,:].to_frame().T
    compare_temp['treatment index'] =  pd.DataFrame(r).T.index
    compare_temp['control index'] = compare['代码'].values
    df_compare = pd.concat([df_compare,compare_temp] , axis = 0)
    
elif divi.shape[0] == 1:
    compare = pd.DataFrame()
    compare_temp = pd.DataFrame()
    compare = divi.iloc[0,:].to_frame().T
    compare_temp['treatment index'] =  pd.DataFrame(r).T.index
    compare_temp['control index'] = compare['代码'].values
    df_compare = pd.concat([df_compare,compare_temp] , axis = 0)

       # else:
#         compare = pd.DataFrame()
#         compare_temp = pd.DataFrame()
#         divi_temp = df_n_c
#         divi_temp['diff_a'] = abs(df_n_c['总资产'] - pd.DataFrame(r).T['总资产'].values)
#         divi_temp['diff_c'] = abs(df_n_c['收盘价'] - pd.DataFrame(r).T['收盘价'].values)
#         divi_temp['diff_d'] = abs(df_n_c['交易天数'] - pd.DataFrame(r).T['交易天数'].values)
#         divi_temp['diff_s'] = abs(df_n_c['流通股合计'] - pd.DataFrame(r).T['流通股合计'].values)
#         divi_temp.sort_values(by=['diff_a','diff_c','diff_d','diff_s'],inplace=True,ascending=True)
#         compare = divi_temp.iloc[0,:].to_frame().T
#         compare_temp['treatment index'] =  pd.DataFrame(r).T.index
#         compare_temp['control index'] = compare['代码'].values
#         df_compare = pd.concat([df_compare,compare_temp] , axis = 0)
#         divi.sort_values(by=['diff_a'],inplace=True,ascending=True)
#         divi.reset_index(inplace = True)
#         divi.reset_index(inplace = True)
#         divi['rank_a'] = divi['level_0']
#         divi = divi.drop('level_0',axis=1)

#         divi.sort_values(by=['diff_c'],inplace=True,ascending=True)
#         divi.reset_index(inplace = True)
#         divi = divi.drop('level_0',axis=1)
#         divi.reset_index(inplace = True)
#         divi['rank_c'] = divi['level_0']
#         divi = divi.drop('level_0',axis=1)

#         divi.sort_values(by=['diff_d'],inplace=True,ascending=True)
#         divi.reset_index(inplace = True)
#         divi = divi.drop('level_0',axis=1)
#         divi.reset_index(inplace = True)
#         divi['rank_d'] = divi['level_0']
#         divi = divi.drop('level_0',axis=1)

#         divi.sort_values(by=['diff_s'],inplace=True,ascending=True)
#         divi.reset_index(inplace = True)
#         divi = divi.drop('level_0',axis=1)
#         divi.reset_index(inplace = True)
#         divi['rank_s'] = divi['level_0']
#         divi = divi.drop('level_0',axis=1)

#         divi['rank'] = divi['rank_a']+divi['rank_c']+divi['rank_d']+divi['rank_s']
#         divi.sort_values(by=['rank'],inplace=True,ascending=True)

#         compare = divi.iloc[0,:].to_frame().T
#         compare_temp['treatment index'] =  pd.DataFrame(r).T.index
#         compare_temp['control index'] = compare['代码'].values
#         df_compare = pd.concat([df_compare,compare_temp] , axis = 0)
else:
    compare = pd.DataFrame()
    compare_temp = pd.DataFrame()
    divi_temp = df_n_c
    divi_temp['diff_a'] = abs(df_n_c['总资产'] - pd.DataFrame(r).T['总资产'].values)
    divi_temp['diff_c'] = abs(df_n_c['收盘价'] - pd.DataFrame(r).T['收盘价'].values)
    divi_temp['diff_d'] = abs(df_n_c['交易天数'] - pd.DataFrame(r).T['交易天数'].values)
    divi_temp['diff_s'] = abs(df_n_c['流通股合计'] - pd.DataFrame(r).T['流通股合计'].values)
    divi_temp.sort_values(by=['diff_a'],inplace=True,ascending=True)
    divi_temp.reset_index(inplace = True)
    divi_temp.reset_index(inplace = True)
    divi_temp['rank_a'] = divi_temp['level_0']
    divi_temp = divi_temp.drop('level_0',axis=1)

    divi_temp.sort_values(by=['diff_c'],inplace=True,ascending=True)
    divi_temp.reset_index(inplace = True)
    divi_temp = divi_temp.drop('level_0',axis=1)
    divi_temp.reset_index(inplace = True)
    divi_temp['rank_c'] = divi_temp['level_0']
    divi_temp = divi_temp.drop('level_0',axis=1)

    divi_temp.sort_values(by=['diff_d'],inplace=True,ascending=True)
    divi_temp.reset_index(inplace = True)
    divi_temp = divi_temp.drop('level_0',axis=1)
    divi_temp.reset_index(inplace = True)
    divi_temp['rank_d'] = divi_temp['level_0']
    divi_temp = divi_temp.drop('level_0',axis=1)

    divi_temp.sort_values(by=['diff_s'],inplace=True,ascending=True)
    divi_temp.reset_index(inplace = True)
    divi_temp = divi_temp.drop('level_0',axis=1)
    divi_temp.reset_index(inplace = True)
    divi_temp['rank_s'] = divi_temp['level_0']
    divi_temp = divi_temp.drop('level_0',axis=1)

    divi_temp['rank'] = divi_temp['rank_a']+divi_temp['rank_c']+divi_temp['rank_d']+divi_temp['rank_s']
    divi_temp.sort_values(by=['rank'],inplace=True,ascending=True)
    
    compare = divi_temp.iloc[0,:].to_frame().T
    compare_temp['treatment index'] =  pd.DataFrame(r).T.index
    compare_temp['control index'] = compare['代码'].values
    df_compare = pd.concat([df_compare,compare_temp] , axis = 0)
    
df_compare

C:\Users\84678\AppData\Local\Temp\ipykernel_24492\838552251.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Segment_c'][df['收盘价'] == 0] = 0
C:\Users\84678\AppData\Local\Temp\ipykernel_24492\838552251.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Segment_d'][df['交易天数'] == 0] = 0
C:\Users\84678\AppData\Local\Temp\ipykernel_24492\838552251.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Segment_s'].loc[df['流通股

,treatment index,control index
0,430014.NQ,831668.NQ


In [24]:
divi

,代码,总资产_x,收盘价_x,交易天数_x,流通股合计_x,Segment_a,Segment_c,Segment_d,Segment_s,名称,做市首日,做市前200交易日,做市前100交易日,做市后200交易日,收盘价_y,交易天数_y,流通股合计_y,匹配期,总资产_y,做市交易前一日


In [ ]:
# 换手率
# def des(df,day):
#     df_des = pd.DataFrame()
#     for i in range(df.shape[0]):
#         des_temp = pd.DataFrame()
#         r = df.iloc[i]
#         date_a1 = r['做市后1交易日']
#         date_a2 = r['做市后'+str(day)+'交易日']
#         date_b1 = r['做市前1交易日']
#         date_b2 = r['做市前'+str(day)+'交易日']
#         r1 = r.to_frame().T
        
#         a_mean = r1.loc[:,date_a1:date_a2].mean(1).values
#         a_median = r1.loc[:,date_a1:date_a2].median(1).values
#         b_mean = r1.loc[:,date_b2:date_b1].mean(1).values
#         b_median = r1.loc[:,date_b2:date_b1].median(1).values

#         des_temp['b_mean'] = b_mean
#         des_temp['a_mean'] = a_mean
#         des_temp['b_median'] = b_median
#         des_temp['a_median'] = a_median
#         des_temp.index = r1.index
        
#         df_des = pd.concat([df_des,des_temp],0)
#     return df_des

# turn_t_20 = des(nonzero_t,20)
# turn_c_20 = des(nonzero_c,20)
# turn_t_50 = des(nonzero_t,50)
# turn_c_50 = des(nonzero_c,50)
# turn_t_100 = des(nonzero_t,100)
# turn_c_100 = des(nonzero_c,100)

# def gen_diff(turn_20,turn_50,turn_100):
#     turn_diff = pd.DataFrame()   
#     turn_diff['mean_20'] = turn_20['a_mean']-turn_20['b_mean']
#     turn_diff['median_20'] = turn_20['a_median']-turn_20['b_median']
#     turn_diff['mean_50'] = turn_50['a_mean']-turn_50['b_mean']
#     turn_diff['median_50'] = turn_50['a_median']-turn_50['b_median']
#     turn_diff['mean_100'] = turn_100['a_mean']-turn_100['b_mean']
#     turn_diff['median_100'] = turn_100['a_median']-turn_100['b_median']    
#     return turn_diff

# turn_diff_t = gen_diff(turn_t_20,turn_t_50,turn_t_100)
# turn_diff_c = gen_diff(turn_c_20,turn_c_50,turn_c_100)

In [11]:
day_t = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "day_t",index_col = 0)
day_c = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "day_c",index_col = 0)
week_t = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week_t",index_col = 0)
week_c = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week_c",index_col = 0)
day_market = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "day_market",index_col = 0)
week_market = pd.read_excel( 'C:/Users/84678/Desktop/做市商效率/定价效率/df_effi.xlsx',sheet_name = "week_market",index_col = 0)

In [43]:
#计算价格滞后指标
md_log1 = day_market.iloc[1].to_frame().T
md_log2 = day_market.iloc[2].to_frame().T
md_log3 = day_market.iloc[3].to_frame().T
md_log4 = day_market.iloc[4].to_frame().T
marday = day_market.iloc[0].to_frame().T

    
r2_temp = pd.DataFrame()
r = day_t.iloc[1]
date_b1 = r['做市前1交易日']
date_b2 = r['做市前100交易日']
date_a1 = r['做市后1交易日']
date_a2 = r['做市后100交易日']

r1 = r.to_frame().T
ret_b = r1.loc[:,date_b2:date_b1].T
ret_a = r1.loc[:,date_a1:date_a2].T
market_b = marday.loc[:,date_b2:date_b1].T
market_a = marday.loc[:,date_a1:date_a2].T
marday_log1_b = md_log1.loc[:,date_b2:date_b1].T 
marday_log2_b = md_log2.loc[:,date_b2:date_b1].T
marday_log3_b = md_log3.loc[:,date_b2:date_b1].T
marday_log4_b = md_log4.loc[:,date_b2:date_b1].T

marday_log1_a = md_log1.loc[:,date_a1:date_a2].T 
marday_log2_a = md_log2.loc[:,date_a1:date_a2].T
marday_log3_a = md_log3.loc[:,date_a1:date_a2].T
marday_log4_a = md_log4.loc[:,date_a1:date_a2].T
X_a = np.column_stack((market_a,marday_log1_a,marday_log2_a,marday_log3_a,marday_log4_a))
X_b = np.column_stack((market_b,marday_log1_b,marday_log2_b,marday_log3_b,marday_log4_b))

reg_b = sm.OLS(ret_b.astype(float),market_b.astype(float)).fit()
reg_a = sm.OLS(ret_a.astype(float),market_a.astype(float)).fit()
reg_b1 = sm.OLS(ret_b.astype(float),X_b).fit()
reg_a1 = sm.OLS(ret_a.astype(float),X_a).fit()

D1_b = 1 - reg_b.rsquared/reg_b1.rsquared
D1_a = 1 - reg_a.rsquared/reg_a1.rsquared

D1_temp = pd.DataFrame([D1_b,D1_a]).T
D1_temp.columns = ['b','a']

coef_b = reg_b1.params.to_frame().T
coef_a = reg_a1.params.to_frame().T
D2_b=(coef_b['x2']+coef_b['x3']+coef_b['x4']+coef_b['x5'])/(coef_b['x1']+coef_b['x2']+coef_b['x3']+coef_b['x4']+coef_b['x5'])
D2_a=(coef_a['x2']+coef_a['x3']+coef_a['x4']+coef_a['x5'])/(coef_a['x1']+coef_a['x2']+coef_a['x3']+coef_a['x4']+coef_a['x5'])

D2_temp = pd.DataFrame([D2_b,D2_a]).T
D2_temp.columns = ['b','a']

In [46]:
corf_b = abs(coef_b)

In [65]:
testDF = pd.DataFrame([[1,2,3,2,4],[3,2,6,4,3]])

In [74]:
testDF.loc[1,:]

0    3
1    2
2    6
3    4
4    3
Name: 1, dtype: int64

In [82]:
un = '6'

In [ ]:
def get_r2_num(df):
    df_r2 = pd.DataFrame()
    for i in range(df.shape[0]):
        r2_temp = pd.DataFrame()
        r = df.iloc[i]
        date_a1 = r['选择日期后1交易日']
        date_a2 = r['选择日期后100交易日']
        
        r1 = r.to_frame().T
        ret_a = r1.loc[:,date_a1:date_a2].T
        market_a = day_market.loc[:,date_a1:date_a2].T
        reg_a = sm.OLS(ret_a.astype(float),market_a.astype(float)).fit()
        
        r2_temp = pd.DataFrame([reg_a.rsquared]).T
        r2_temp.columns = ['syn']
        
        df_r2 = pd.concat([df_r2,r2_temp],0)
    
    df_r2.index = df.index.values
    df_r2 = df_r2.dropna(how='any')
    return df_r2